In [34]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [31]:
#!pip install xgboost

In [35]:
try:
    df_trabajo_fusion = pd.read_csv("../output/Datos_Fusionadosv2_escalado.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")

In [36]:
# Seleccionar las características y la variable objetivo
X = df_trabajo_fusion.drop(columns=['Tipo_Mantenimiento'])
y = df_trabajo_fusion['Tipo_Mantenimiento']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar los modelos
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForestRegressor': RandomForestRegressor()
}

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse

# Imprimir los resultados
for name, mse in results.items():
    print(f'{name}: Mean Squared Error = {mse}')

LinearRegression: Mean Squared Error = 0.24155428354999575
Ridge: Mean Squared Error = 0.24155282320810162
Lasso: Mean Squared Error = 0.25007111301252094
RandomForestRegressor: Mean Squared Error = 0.0548101958814666


In [ ]:
# Inicializar el modelo XGBoost
xgb_model = XGBClassifier()

# Definir la cuadrícula de hiperparámetros para ajustar
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo encontrado por GridSearchCV
best_xgb_model = grid_search.best_estimator_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred_best = best_xgb_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
classification_report_best = classification_report(y_test, y_pred_best)

# Imprimir las métricas de evaluación del mejor modelo
print(f'Accuracy: {accuracy_best}')
print(f'F1-Score: {f1_best}')
print('Classification Report:')
print(classification_report_best)

Accuracy: 0.9472626820693119
F1-Score: 0.951545916012921
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94       957
           1       0.91      1.00      0.95      1034

    accuracy                           0.95      1991
   macro avg       0.95      0.95      0.95      1991
weighted avg       0.95      0.95      0.95      1991



La conclusiones que es que los modelos de regresion no son efecaces para la problematica de estos datos no lineales y complejos, en cambio XGBoost, muesta un buen balance y un grado de precision muy aceptable para estos datos no lineales, nos genera la duda de que quizas este sobreajustado